# Controlo da rota de 3 navios num lago infinito

In [2]:
from z3 import *
import numpy as np
from math import cos, sin

In [32]:
VEL_INIT, VEL_HIGH, VEL_LOW = 0, 10, 1 # velocidade em m/s
THETA = 15 # angulo de viragem em graus
TAU = 1 # intervalo de tempo entre estados temporais consecutivos em ??? (unidade a determinar)
SAFE_DIST = 50 # distancia de seguranca entre barcos em m

GAUSS_DP = 80 # desvio padrao da gaussiana da inicializacao da posicao em m

In [ ]:
def declare_boat(boat_id):
    """
    A velocidade do state determina qual o seu modo:
        - INIT: vel = VEL_INIT
        - SAFE: vel = VEL_HIGH
        - IMMINENT COLLISION: vel = VEL_LOW
    """
    
    state = {
        "vel": Int(f"b{boat_id}_v"),
        "pos": {"x": Real(f"b{boat_id}_x"), "y": Real(f"b{boat_id}_y")},
        "ang": Int(f"b{boat_id}_a"),
        "time": Real(f"b{boat_id}_t")
    }
    
    return state


def init_boat(state):
    # cond_a = Or([state["ang"] == i*THETA for i in range(360/15)])
    
    # Randomly defined angle
    random_angle = np.random.shuffle([i*THETA for i in range(360/15)])[0]
    cond_a = state["ang"] == random_angle
    
    # Randomly defined position according to gaussian
    cond_x = state["pos"]["x"] == GAUSS_DP * np.random.randn()
    cond_y = state["pos"]["y"] == GAUSS_DP * np.random.randn()
    
    # Deterministic initial definitions
    cond_v = state["vel"] == VEL_INIT
    cond_t = state["time"] == 0
    
    return And(cond_a, cond_v, cond_t)


def untimed_to_imminent(state):
    return state["vel"] == VEL_LOW


def untimed_to_safe(state):
    return state["vel"] == VEL_HIGH


def timed_safe(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    return And(cond_t, cond_x, cond_y)


def timed_imminent(state_ant, state_atual):
    # Definir evolucao do tempo
    cond_t = state_atual["time"] == state_ant["time"] + TAU
    
    # Definir evolucao da abcissa
    dx = state_ant["vel"] * cos(state_ant["ang"]) * TAU
    cond_x = state_atual["pos"]["x"] == state_ant["pos"]["x"] + dx
    
    # Definir evolucao da ordenada
    dy = state_ant["vel"] * sin(state_ant["ang"]) * TAU
    cond_y = state_atual["pos"]["y"] == state_ant["pos"]["y"] + dy
    
    # Definir evolucao da rota
    theta_pos = state_atual["ang"] == state_ant["ang"] + THETA
    theta_neg = state_atual["ang"] == state_ant["ang"] - THETA
    cond_a = Or(theta_pos, theta_neg)
    
    return And(cond_t, cond_x, cond_y, cond_a)


def imminent_collision():
    

In [36]:
def declare(i, m=0):
    mode = {
        "b1": {"v": Int(f"t{i}m{m}_b1_v"), "a": Int(f"t{i}m{m}_b1_a")},
        "b2": {"v": Int(f"t{i}m{m}_b2_v"), "a": Int(f"t{i}m{m}_b2_a")},
        "b3": {"v": Int(f"t{i}m{m}_b3_v"), "a": Int(f"t{i}m{m}_b3_a")}
    }
    
    state = {
        "b1": {"x": Real(f"t{i}m{m}_b1_x"), "y": Real(f"t{i}m{m}_b1_y")},
        "b2": {"x": Real(f"t{i}m{m}_b2_y"), "y": Real(f"t{i}m{m}_b2_y")},
        "b3": {"x": Real(f"t{i}m{m}_b3_y"), "y": Real(f"t{i}m{m}_b3_y")}
    }
    
    trace = {
        "mode": mode,
        "state": state,
        "time": Int(f"t{i}"),
        "mode_num": m
    }
    
    return trace


def init(trace):
    conds = []
    num_boats = len(trace["mode"])
    
    # Randomly defined initial angles
    angles = [i*THETA for i in range(int(360/15))]
    np.random.shuffle(angles)
    random_angles = [angles[0] for _ in range(num_boats)]
    for i, b in enumerate(trace["mode"]):
        conds.append(trace["mode"][b]["v"] == VEL_HIGH)
        conds.append(trace["mode"][b]["a"] == random_angles[i])
    
    # Randomly defined position according to gaussian
    for i in range(len(trace["state"])):
        conds.append(trace["state"][f"b{i+1}"]["x"] == GAUSS_DP * np.random.randn())
        conds.append(trace["state"][f"b{i+1}"]["y"] == GAUSS_DP * np.random.randn())
    
    # Set initial time to 0
    conds.append(trace["time"] == 0)
    
    return And(conds)


def untimed_to_imminent(t, trace_mant, boat_id1, boat_id2):
    # NOTA: Possivelmente teremos de colocar uma condicao Or para os dois casos +-THETA
    # Criar o novo estado atual
    trace_matual = declare(t, trace_mant["mode_num"]+1)
    num_boats = len(trace_mant["mode"])
    
    # Condicao z3 do modo
    conds = []
    boat_ids = [i for i in range(1, num_boats+1) if (i!=boat_id1 and i!=boat_id2)]
    conds.append(trace_matual["mode"][f"b{boat_id1}"]["v"] == VEL_LOW)
    conds.append(trace_matual["mode"][f"b{boat_id2}"]["v"] == VEL_LOW)
    conds.append(trace_matual["mode"][f"b{boat_id1}"]["a"] == trace_mant["mode"][f"b{boat_id1}"]["a"] + THETA)
    conds.append(trace_matual["mode"][f"b{boat_id2}"]["a"] == trace_mant["mode"][f"b{boat_id2}"]["a"] + THETA)
    for boat_id in boat_ids:
        conds.append(trace_matual["mode"][f"b{boat_id}"]["v"] == trace_mant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_matual["mode"][f"b{boat_id}"]["a"] == trace_mant["mode"][f"b{boat_id}"]["a"])
    
    # Condicao z3 do estado
    for boat_id in range(1, num_boats+1):
        conds.append(trace_matual["state"][f"b{boat_id}"]["x"] == trace_mant["state"][f"b{boat_id}"]["x"])
        conds.append(trace_matual["state"][f"b{boat_id}"]["y"] == trace_mant["state"][f"b{boat_id}"]["y"])
    conds.append(trace_matual["time"] == trace_mant["time"])
    
    return And(conds), trace_matual


def untimed_to_safe(t, trace_mant, boat_id1):
    # Criar o novo estado atual
    trace_matual = declare(t, trace_mant["mode_num"]+1)
    num_boats = len(trace_mant["mode"])
    
    # Condicao z3 do modo
    conds = []
    boat_ids = [i for i in range(1, num_boats+1) if i!=boat_id1]
    conds.append(trace_matual["mode"][f"b{boat_id1}"]["v"] == VEL_HIGH)
    conds.append(trace_matual["mode"][f"b{boat_id1}"]["a"] == trace_mant["mode"][f"b{boat_id1}"]["a"])
    for boat_id in boat_ids:
        conds.append(trace_matual["mode"][f"b{boat_id}"]["v"] == trace_mant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_matual["mode"][f"b{boat_id}"]["a"] == trace_mant["mode"][f"b{boat_id}"]["a"])
    
    # Condicao z3 do estado
    for boat_id in range(1, num_boats+1):
        conds.append(trace_matual["state"][f"b{boat_id}"]["x"] == trace_mant["state"][f"b{boat_id}"]["x"])
        conds.append(trace_matual["state"][f"b{boat_id}"]["y"] == trace_mant["state"][f"b{boat_id}"]["y"])
    conds.append(trace_matual["time"] == trace_mant["time"])
    
    return And(conds), trace_matual          
    

def timed(trace_ant, trace_atual):
    # Definir evolucao do tempo
    conds = []
    num_boats = len(trace_ant["mode"])
    conds.append(trace_atual["time"] == trace_ant["time"] + TAU)
    
    # Definir evolucao do estado
    for boat_id in range(1, num_boats+1):
        # Definir evolucao do modo do barco boat_id
        conds.append(trace_atual["mode"][f"b{boat_id}"]["v"] == trace_ant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_atual["mode"][f"b{boat_id}"]["a"] == trace_ant["mode"][f"b{boat_id}"]["a"])
        
        # Calculo do incremento da posicao
        dx = trace_ant["mode"][f"b{boat_id}"]["v"] * cos(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
        dy = trace_ant["mode"][f"b{boat_id}"]["v"] * sin(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
        
        # Condicao z3 da posicao
        conds.append(trace_atual["state"][f"b{boat_id}"]["x"] == trace_ant["state"][f"b{boat_id}"]["x"] + dx)
        conds.append(trace_atual["state"][f"b{boat_id}"]["y"] == trace_ant["state"][f"b{boat_id}"]["y"] + dy)
    
    return And(cond_t, cond_x, cond_y)


def abs(x):
    return If(x >= 0, x, -x)


def switch_no_collision(trace, boat_id1, boat_id2):
    cond_x = abs(trace["state"][f"b{boat_id1}"]["x"] - trace["state"][f"b{boat_id2}"]["x"]) <= SAFE_DIST
    cond_y = abs(trace["state"][f"b{boat_id1}"]["y"] - trace["state"][f"b{boat_id2}"]["y"]) <= SAFE_DIST
    
    return And(cond_x, cond_y)


def trans(t, trace_ant, trace_atual):
    num_boats = len(trace_ant["mode"])
    
    # Untimed transitions
    conds = []
    for i in range(1, num_boats+1):
        switch_conds = []
        for j in range(1, num_boats+1):
            if i != j:
                # Untimed to imminent
                cond = switch_no_collision(trace_ant, i, j)
                cond1, trace_ant = untimed_to_imminent(t, trace_ant, i, j)
                conds.append(Implies(Not(cond), cond1))
                
                switch_conds.append(cond)
        
        # Untimed to safe
        cond, trace_ant = untimed_to_safe(t, trace_ant, i)
        conds.append(Implies(And(switch_conds), cond))
            
    # Timed transitions
    conds.append(timed(trace_ant, trace_atual))
    
    return And(conds)

In [22]:
solver = Solver()
trace = declare(0)
print(trace)
conds, trace = untimed_to_imminent(0, trace, 1, 2)
solver.add(conds)
print(solver.assertions)

{'mode': {'b1': {'v': t0m0_b1_v, 'a': t0m0_b1_a}, 'b2': {'v': t0m0_b2_v, 'a': t0m0_b2_a}, 'b3': {'v': t0m0_b3_v, 'a': t0m0_b3_a}}, 'state': {'b1': {'x': t0m0_b1_x, 'y': t0m0_b1_y}, 'b2': {'x': t0m0_b2_y, 'y': t0m0_b2_y}, 'b3': {'x': t0m0_b3_y, 'y': t0m0_b3_y}}, 'time': t0, 'mode_num': 0}
<bound method Solver.assertions of [And(t0m1_b1_v == 1,
     t0m1_b2_v == 1,
     t0m1_b1_a == t0m0_b1_a + 15,
     t0m1_b2_a == t0m0_b2_a + 15,
     t0m1_b3_v == t0m0_b3_v,
     t0m1_b3_a == t0m0_b3_a,
     t0m1_b1_x == t0m0_b1_x,
     t0m1_b1_y == t0m0_b1_y,
     t0m1_b2_y == t0m0_b2_y,
     t0m1_b2_y == t0m0_b2_y,
     t0m1_b3_y == t0m0_b3_y,
     t0m1_b3_y == t0m0_b3_y,
     t0 == t0)]>


In [37]:
def time_evolution(declare, init, trans, k):
    # Criar o solver e o traço de estados temporais
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}

    # Inicializar o primeiro estado temporal
    solver.add(init(trace[0]))

    # Fazer a transição de estados
    for i in range(1, k):
        solver.add(trans(i-1, trace[i-1], trace[i]))

    if solver.check() == sat:
        m = solver.model()
        print(m)
        r = m
    else:
        r = None
        
    return r

time_evolution(declare, init, trans, 10)

TypeError: must be real number, not ArithRef